<a href="https://colab.research.google.com/github/ericjl12264/Device/blob/master/Yolov8%20Export%20tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install PyTorch (Colab usually has it, but this ensures everything is up-to-date)
!pip install torch

# Install Ultralytics package (YOLOv8 uses this structure)
!pip install ultralytics

import torch
import os
import sys

# --- Configuration ---
# Set the path to your model file.
# You must upload SMDbest2.pt to the Colab environment first.
MODEL_PATH = '/content/SMDbest2.pt'

if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Please upload your model file to Colab and ensure the path is correct: {MODEL_PATH}")

print(f"Loading model from: {MODEL_PATH}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.4 MB/s eta 0:00:00
Loading model from: /content/SMDbest2.pt


In [3]:
def investigate_yolo_model(model_path):
    """Loads a PyTorch .pt file and attempts to determine if it's YOLOv5 or YOLOv8."""

    # Load the checkpoint file
    try:
        # Load the PyTorch model checkpoint
        # Setting map_location='cpu' ensures it loads even if you don't have a GPU
        # Setting weights_only=False to address the error with unsupported globals from ultralytics
        ckpt = torch.load(model_path, map_location='cpu', weights_only=False)
    except Exception as e:
        print(f"❌ Error loading checkpoint file: {e}")
        return

    print("✅ Checkpoint file loaded successfully.")

    # --- Inspection 1: Check Ultralytics Metadata ---
    # Ultralytics often saves metadata in a specific key.
    if 'meta' in ckpt:
        meta_data = ckpt['meta']
        print("\n🔎 Inspection 1: Ultralytics Metadata Found")

        # YOLOv8 models, when saved, usually have the 'ultralytics' key in the metadata
        if 'ultralytics_version' in meta_data:
            print(f"   -> Ultralytics Version: {meta_data['ultralytics_version']}")
            print("   -> LIKELY **YOLOv8** (or a very recent YOLOv5/Ultralytics version).")

        # Older YOLOv5 models might contain specific run info directly in the checkpoint root
        elif 'train_args' in ckpt:
            if 'hyp' in ckpt['train_args']:
                print("   -> Contains 'hyp' and 'train_args'. LIKELY **YOLOv5**.")
            else:
                print("   -> Contains 'train_args' but lacks typical YOLOv5 hyperparams.")

    # --- Inspection 2: Check for Model Architecture Keys ---
    # YOLOv5 architecture often includes an 'anchors' key in the model state dictionary,
    # which is explicitly removed in the anchor-free YOLOv8 architecture.
    if 'model' in ckpt:
        model_state_dict = ckpt['model'].state_dict() if hasattr(ckpt['model'], 'state_dict') else None

        if model_state_dict:
            print("\n🔎 Inspection 2: Model Architecture Check")

            # Check for the presence of the 'anchors' key
            if any('anchor' in k.lower() for k in model_state_dict.keys()):
                print("   -> Found layers related to **anchors** (e.g., 'model.24.anchor_grid').")
                print("   -> **CONFIRMED: YOLOv5 Architecture** (Anchor-based).")
            else:
                # If no anchor keys are found, it's more likely to be anchor-free.
                print("   -> Did NOT find typical 'anchors' keys in model state.")
                print("   -> **HIGHLY LIKELY: YOLOv8 Architecture** (Anchor-free).")
        else:
            print("\n🔎 Inspection 2: Could not extract model state dictionary.")

    else:
        print("\n🔎 Inspection 2: No 'model' key found in the checkpoint. File structure is unusual.")

# Run the investigation function
investigate_yolo_model(MODEL_PATH)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Checkpoint file loaded successfully.

🔎 Inspection 2: Model Architecture Check
   -> Did NOT find typical 'anchors' keys in model state.
   -> **HIGHLY LIKELY: YOLOv8 Architecture** (Anchor-free).


In [1]:


# Install the official Ultralytics package for YOLOv8 (should be compatible with the above)
!pip install ultralytics

import os
from ultralytics import YOLO
import yaml

# Add current directory to sys.path to resolve potential TensorFlow import issues
import sys
if '.' not in sys.path:
    sys.path.insert(0, '.')

import tensorflow as tf

print(f"Ultralytics installed. TensorFlow version: {tf.__version__}")

Ultralytics installed. TensorFlow version: 2.19.0


In [3]:
# --- Configuration ---
# Assuming the file is uploaded to the Colab root
MODEL_PATH = '/content/SMDbest2.pt'
CALIBRATION_DIR = 'calibration_images' # Folder with representative images
CALIB_YAML_PATH = 'calib_data.yaml'    # Temporary YAML file for calibration
TFLITE_OUTPUT_PATH = 'SMDbest2_quantized_int8.tflite'
INPUT_IMAGE_SIZE = 640

# Verify files exist
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Please upload the PyTorch weights file to Colab: {MODEL_PATH}")

# Create the calibration directory if it doesn't exist
os.makedirs(CALIBRATION_DIR, exist_ok=True)

# If the directory is empty, a warning should be displayed, but it won't stop execution
if not os.listdir(CALIBRATION_DIR):
    print(f"Warning: The '{CALIBRATION_DIR}' folder is empty. Please populate it with representative images for INT8 quantization.")

# Create the minimal YAML file for YOLOv8 export to find the calibration data
# This YAML assumes your images are in the '/content/calibration_images' folder
data = {
    'path': '/content/',
    'train': CALIBRATION_DIR, # Use the calibration images as the 'train' set for export calibration
    'val': CALIBRATION_DIR,
    'names': {i: f'class_{i}' for i in range(80)} # Replace 80 with your actual number of classes
}

# Write the temporary YAML file
with open(CALIB_YAML_PATH, 'w') as f:
    yaml.dump(data, f)

print(f"Calibration data YAML created at: {CALIB_YAML_PATH}")

Calibration data YAML created at: calib_data.yaml


In [4]:
print("--- Starting YOLOv8 PyTorch to TFLite INT8 Export ---")

try:
    # 1. Load the YOLOv8 model from the .pt file
    model = YOLO(MODEL_PATH)

    # 2. Perform the export using the ultralytics export function
    # - format='tflite' tells it to use the TFLite Converter.
    # - int8=True enables full integer quantization.
    # - data=CALIB_YAML_PATH provides the path to the representative dataset.
    # - imgsz=INPUT_IMAGE_SIZE sets the input resolution.
    exported_model = model.export(
        format='tflite',
        int8=True,
        data=CALIB_YAML_PATH,
        imgsz=INPUT_IMAGE_SIZE
    )

    # The output is saved to the same directory as the weights, usually
    # /content/SMDbest2.tflite. We rename it for clarity.

    # Check the actual name created by export (it usually appends _int8.tflite)
    # We will look for the first .tflite file created in the current directory
    tflite_files = [f for f in os.listdir('.') if f.endswith('.tflite')]

    if tflite_files:
        final_tflite_path = tflite_files[0]
        # Rename the file for consistency
        os.rename(final_tflite_path, TFLITE_OUTPUT_PATH)

        print(f"\n✅ TFLite INT8 model saved successfully to: {TFLITE_OUTPUT_PATH}")

        # Optional: Download the file automatically
        from google.colab import files
        files.download(TFLITE_OUTPUT_PATH)
    else:
        print("❌ Export successful, but could not locate the new .tflite file.")


except Exception as e:
    print(f"\n❌ ERROR during YOLOv8 TFLite export: {e}")
    print("Ensure all required files are uploaded and the number of classes in the YAML is correct.")

--- Starting YOLOv8 PyTorch to TFLite INT8 Export ---
Ultralytics 8.3.229 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/content/SMDbest2.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (5.9 MB)
requirements: Ultralytics requirements ['sng4onnx>=1.0.1', 'onnx_graphsurgeon>=0.3.26', 'ai-edge-litert>=1.2.0', 'onnx>=1.12.0,<=1.19.1', 'onnx2tf>=1.26.3', 'onnxslim>=0.1.71', 'onnxruntime'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 20 packages in 7.01s
Prepared 12 packages in 6.82s
Uninstalled 2 packages in 46ms
Installed 12 packages in 427ms
 + ai-edge-litert==2.0.3
 + backports-strenum==1.3.1
 + colorama==0.4.6
 + coloredlogs==15.0.1
 + humanfriendly==10.0
